In [1]:
%run base.ipynb

<h1>System Inertia Matrix example</h1>

In [2]:
inertia_mat = uv_body.body_inertia_matrix()
M_func = Function('M_b', [m, I_o, z_g, decoupled_added_m, coupled_added_m], [inertia_mat]) # for numerical & symbolic use

# M_func.save('M.casadi')

In [3]:
# example usage of inertia mat
M_mat = np.array(M_func(blue.m, blue.Io, blue.rg[2], blue.added_m, blue.coupl_added_m))
M_mat

array([[17.  ,  0.  ,  0.  ,  0.  ,  0.23,  0.  ],
       [ 0.  , 24.2 ,  0.  , -0.23,  0.  ,  0.  ],
       [ 0.  ,  0.  , 26.07,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.23,  0.  ,  0.28,  0.  ,  0.  ],
       [ 0.23,  0.  ,  0.  ,  0.  ,  0.28,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.28]])

In [4]:
nn_UVparam = nnVehicleParams(blue.m, 
                             torch.from_numpy(blue.Ib_b), 
                             torch.from_numpy(blue.rg), 
                             torch.from_numpy(blue.rb), 
                             torch.from_numpy(blue.MA), 
                             torch.from_numpy(blue.linear_dc), 
                             torch.from_numpy(blue.quadratic_dc), 
                             blue.W, 
                             blue.B)
nndynamics = nnVehicleDynamics(nn_UVparam, False)

In [5]:
nn_M_mat = nndynamics.body_inertia_matrix()
nn_M_mat

tensor([[17.0000,  0.0000,  0.0000, -0.0000,  0.2300, -0.0000],
        [ 0.0000, 24.2000,  0.0000, -0.2300, -0.0000,  0.0000],
        [ 0.0000,  0.0000, 26.0700,  0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.2300,  0.0000,  0.2800,  0.0000,  0.0000],
        [ 0.2300,  0.0000, -0.0000,  0.0000,  0.2800,  0.0000],
        [-0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.2800]],
       dtype=torch.float64)

In [6]:
nn_M_mat.T == nn_M_mat

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True]])

In [7]:
M_mat.T == M_mat

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

In [8]:
inertia_ned_mat = uv_ned.ned_euler_inertia_matrix()
Onb = vertcat(thet, phi, psi)
M_ned_func = Function('M_ned', [m, I_o, z_g, Onb, decoupled_added_m, coupled_added_m], [inertia_ned_mat]) # for numerical & symbolic use

In [9]:
# example usage of inertia mat in ned
Onb_v = np.array([0.13,0.17,0.1])
np.array(M_ned_func(blue.m, blue.Io, blue.rg[2], Onb_v,  blue.added_m, blue.coupl_added_m))

array([[ 1.72303360e+01, -7.45090577e-01,  1.18405662e+00,
         1.76115723e-02,  2.23648786e-01,  3.57838329e-02],
       [-7.45090577e-01,  2.41747103e+01, -1.91929773e-01,
        -2.26055609e-01,  2.24397276e-02,  3.31239544e-02],
       [ 1.18405662e+00, -1.91929773e-01,  2.58649536e+01,
        -3.85835972e-02, -2.93860503e-02,  0.00000000e+00],
       [ 1.76115723e-02, -2.26055609e-01, -3.85835972e-02,
         2.80000000e-01,  0.00000000e+00, -3.62975599e-02],
       [ 2.23648786e-01,  2.24397276e-02, -2.93860503e-02,
         0.00000000e+00,  2.80000000e-01,  0.00000000e+00],
       [ 3.57838329e-02,  3.31239544e-02,  0.00000000e+00,
        -3.62975599e-02,  0.00000000e+00,  2.80000000e-01]])

<h1>Coriolis Matrix example</h1>

In [10]:
coriolis_mat = uv_body.body_coriolis_centripetal_matrix()
C_func = Function('C_b', [m, I_o, z_g, decoupled_added_m, coupled_added_m, x_nb, v_c], [coriolis_mat]) # for numerical & symbolic use

In [11]:
v_r_0 = np.array([0.1 ,0.2,0.1,0.1,0.1,0.5]) # example velocity vector
C = np.array(C_func(blue.m, blue.Io, blue.rg[2], blue.added_m, blue.coupl_added_m, v_r_0, 
                    np.array([0.0 ,0.0,0.0,0.0,0.0,0.0])))
C

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  2.607, -4.817],
       [ 0.   ,  0.   ,  0.   , -2.607,  0.   ,  1.723],
       [ 0.   ,  0.   ,  0.   ,  4.817, -1.723,  0.   ],
       [ 0.   ,  2.607, -4.817,  0.   ,  0.14 , -0.051],
       [-2.607,  0.   ,  1.723, -0.14 ,  0.   , -0.018],
       [ 4.817, -1.723,  0.   ,  0.051,  0.018,  0.   ]])

In [12]:
nnC = nndynamics.body_coriolis_centripetal_matrix(torch.from_numpy(v_r_0))
nnC

tensor([[ 0.0000,  0.0000,  0.0000, -0.0000,  2.6070, -4.8170],
        [ 0.0000,  0.0000,  0.0000, -2.6070, -0.0000,  1.7230],
        [ 0.0000,  0.0000,  0.0000,  4.8170, -1.7230, -0.0000],
        [-0.0000,  2.6070, -4.8170, -0.0000,  0.1400, -0.0510],
        [-2.6070, -0.0000,  1.7230, -0.1400, -0.0000, -0.0180],
        [ 4.8170, -1.7230, -0.0000,  0.0510,  0.0180, -0.0000]],
       dtype=torch.float64)

In [13]:
-C.T == C

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

In [14]:
-nnC.T == nnC

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True]])

<h1>Restoring Forces</h1>

In [15]:
res_G = uv_body.body_restoring_vector()
res_G_func = Function('G_n', [z, eul, W, B, r_g, r_b, B_eps], [res_G]) # for numerical & symbolic use

In [16]:
blue_eul = np.array([0.1, 0.2, 0.3]) # example euler angles
res_G_func(blue.below_surface, blue_eul, blue.W, blue.B, blue.rg, blue.rb, blue.B_eps)

DM([-0.131453, 0.0647397, 0.645238, 0.220764, 0.448258, -0])

In [17]:
nndynamics.body_restoring_vector(torch.from_numpy(blue_eul))

tensor([-0.3944,  0.1942,  1.9357,  0.2208,  0.4483, -0.0000],
       dtype=torch.float64)

In [18]:
# # c , cpp or matlab code generation for forward dynamics
# res_G_func.generate("res_G_func.c")
# os.system(f"gcc -fPIC -shared res_G_func.c -o libg.so")

In [19]:
damping = uv_body.body_damping_matrix()
damping_func = Function('G_n', [x_nb, v_c, linear_dc, quadratic_dc], [damping]) # for numerical & symbolic use

In [20]:
damping_func(v_r_0, blue.v_flow, blue.linear_dc, blue.quadratic_dc)

DM(
[[5.848, 00, 00, 00, 00, 00], 
 [00, 10.552, 00, 00, 00, 00], 
 [00, 00, 8.879, 00, 00, 00], 
 [00, 00, 00, 0.225, 00, 00], 
 [00, 00, 00, 00, 0.225, 00], 
 [00, 00, 00, 00, 00, 0.845]])

In [ ]:
nndynamics.body_damping_matrix(torch.from_numpy(v_r_0))

tensor([[ 5.8480, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
        [-0.0000, 10.5520, -0.0000, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000,  8.8790, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000,  0.2250, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000,  0.2250, -0.0000],
        [-0.0000, -0.0000, -0.0000, -0.0000, -0.0000,  0.8450]],
       dtype=torch.float64)

In [22]:
bias_force = uv_body.get_bias()
bias_force_func = Function('Bias_b', [z, eul, x_nb ,sim_p, f_ext], [bias_force])
# bias_force_func.save('C.casadi')

# # c , cpp or matlab code generation for dynamics instance
# bias_force_func.generate("C_uv_vec.c")
# os.system(f"gcc -fPIC -shared C_uv_vec.c -o libBias_vehicle.so")

In [23]:
blue_f_ext = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # example external forces
bias_force_func(blue.below_surface, blue_eul, v_r_0, blue.sim_params, blue_f_ext)

DM([-1.69445, 2.77594, 1.84254, 0.271464, 0.359358, 0.5665])

In [25]:
nndynamics.get_bias(torch.from_numpy(blue_eul),
                    torch.from_numpy(v_r_0),
                    torch.from_numpy((blue_f_ext)))

tensor([-1.9574,  2.9054,  3.1330,  0.2715,  0.3594,  0.5665],
       dtype=torch.float64)

In [13]:
M_mat_func = Function('M_b', [sim_p], [inertia_mat]) # for numerical & symbolic use
# M_mat_func.save('M.casadi')

# # c , cpp or matlab code generation for dynamics instance
# M_mat_func.generate("M_uv_matrix.c")
# os.system(f"gcc -fPIC -shared M_uv_matrix.c -o libMass_vehicle.so")